In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
try:
    import openai
except:
    !pip install openai
    import openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 4.6 MB/s eta 0:00:00


In [3]:
os.environ["OPENAI_API_KEY"] = "REDACTED"  #api_key for ChatGPT LLM


In [4]:
print ("Got to here")

Got to here


In [5]:
# Query the model
#repsonse = openai.completions.create(
#  model="gpt-3.5-turbo-instruct",
#  prompt="Closest words to 'religion':",
#  max_tokens=50,
#  stop="\n"
#)

# Print the response
#print(response.choices[0].text.strip())

In [6]:
#print(response) #.choices[0].text.strip())

In [7]:
print ("Got to here too")

Got to here too


In [8]:
def jaccard_similarity(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    print (set2-set1)   # the words in list2 that do not exist in list1
    return intersection / union
    #print (set2-set1)

In [9]:
word2vec_list = ["Office of the registrar", "university records office", "student registration office", "Dean's office", "Admissions office", "students records office", "academic office", "enrolment office", "academic affairs office", "education office"]
llm_list = ["Office of the registrar", "Bizarre", "education office"]


In [10]:
jaccard_similarity(word2vec_list,llm_list)

{'Bizarre'}


0.18181818181818182